In [1]:
from math import sin, cos, log
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [2]:
# model parameters:
g = 9.8      # gravity in m s^{-2}
v_t = 4.9    # trim velocity in m s^{-1}   
C_D = 1/5.0  # drag coefficient --- or D/L if C_L=1
C_L = 1.0    # for convenience, use C_L = 1

### set initial conditions ###
v0 = 6.5     # start at the trim velocity (or add a delta)
theta0 = -0.1 # initial angle of trajectory
x0 = 0.0     # horizotal position is arbitrary
y0 = 2.0     # initial altitude

In [7]:
def f(u):
    """Returns the right-hand side of the phugoid system of equations.
    
    Parameters
    ----------
    u : array of float
        array containing the solution at time n.
        
    Returns
    -------
    dudt : array of float
        array containing the RHS given u.
    """
    
    v = u[0]
    theta = u[1]
    x = u[2]
    y = u[3]
    return np.array([-g*sin(theta) - C_D/C_L*g/v_t**2*v**2,
                      -g*cos(theta)/v + g/v_t**2*v,
                      v*cos(theta),
                      v*sin(theta)])


def euler_step(u, f, dt):
    """Returns the solution at the next time-step using Euler's method.
    
    Parameters
    ----------
    u : array of float
        solution at the previous time-step.
    f : function
        function to compute the right hand-side of the system of equation.
    dt : float
        time-increment.
    
    Returns
    -------
    u_n_plus_1 : array of float
        approximate solution at the next time step.
    """
    
    return u + dt * f(u)


def get_diffgrid(u_current, u_fine, dt):
    """Returns the difference between one grid and the fine one using L-1 norm.
    
    Parameters
    ----------
    u_current : array of float
        solution on the current grid.
    u_finest : array of float
        solution on the fine grid.
    dt : float
        time-increment on the current grid.
    
    Returns
    -------
    diffgrid : float
        difference computed in the L-1 norm.
    """
    
    N_current = len(u_current[:,0])
    N_fine = len(u_fine[:,0])
   
    grid_size_ratio = np.ceil(N_fine/N_current)
    
    diffgrid = dt * np.sum( numpy.abs(\
            u_current[:,2]- u_fine[::grid_size_ratio,2])) 
    
    return diffgrid

In [8]:
def rk2_step(u, f, dt):
    """ Returns the solution at the next time-step using 2nd-order Runge-Kutta.
    
    Parameters
    ----------
    u : array of float
        solution at the provious time step
    f : function
        function to compute the right hand-side of the system of equations
    dt : float
        time-increment.
    
    Returns
    -------
    u_n_plus1 : array of float
        solution at the next time step
    """
    u_star = u + 0.5*dt*f(u)
    return u + dt*f(u_star)

In [11]:
# set time-increment and discretize the time
T = 15.           # final time
dt = 0.01         # set time-increment
N = int(T/dt) + 1 # number of time-steps

# initial conditions
u_euler = np.empty((N, 4))
u_rk2 = np.empty((N, 4))

# initialize the array containting the solution for each time step
u_euler[0] = np.array([v0, theta0, x0, y0])
u_rk2[0] = np.array([v0, theta0, x0, y0])

# use for loop to call the function rd2_step()
for n in range(N-1):
    u_euler[n+1] = euler_step(u_euler[n], f, dt)
    u_rk2[n+1] = rk2_step(u_rk2[n], f, dt)

In [21]:
x_euler = u_euler[:,2]
y_euler = u_euler[:,3]
x_rk2 = u_rk2[:,2]
y_rk2 = u_rk2[:,3]

In [22]:
np.where(y_euler < 0)[0][0]

310

In [23]:
# get index of element of y where altitude becomes negative
idx_negative_euler = np.where(y_euler < 0.0)[0]
if len(idx_negative_euler)==0:
    idx_ground_euler = N-1
    print('Euler integration has not touched ground yet!')
else:
    idx_ground_euler = idx_negative_euler[0]

idx_negative_rk2 = np.where(y_rk2 < 0.0)[0]
if len(idx_negative_rk2)==0:
    idx_ground_rk2 = N-1
    print('Runge-Kutta integration has not touched ground yet!')
else: idx_ground_rk2 = idx_negative_rk2[0]